# **Bioinformatics Project - Computational Drug Discovery: Bioactivity Prediction App**

Chanin Nantasenamat

[*'Data Professor' YouTube channel*](http://youtube.com/dataprofessor)

In this Jupyter notebook, we will be building a Streamlit web app for predicting bioactivity of compounds against acetylcholinesterase.

---

## **1. Import libraries**

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski